In [1]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd  
import numpy as np
import pickle
import nltk
import string
import matplotlib.pyplot as plt
import random
import csv
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline
import re


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))


import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\godbo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\godbo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset = pd.read_csv("new_reviews.csv")


vocab_size = 50000
embedding_dim = 256
max_length = 1000
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

x_train, x_test, y_train, y_test = train_test_split(dataset['reviews'], dataset['ratings'], test_size=0.1)

In [3]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:20])


{'<OOV>': 1,
 "'": 2,
 'x83': 3,
 'x82ã': 4,
 'x82': 5,
 'x83ã': 6,
 "'rated": 7,
 'n': 8,
 "0'": 9,
 "'place'": 10,
 "'good'": 11,
 "'food'": 12,
 "'4": 13,
 "'5": 14,
 "n'": 15,
 "'chicken'": 16,
 "'3": 17,
 "'service'": 18,
 "'ordered'": 19,
 "'great'": 20}

In [4]:
train_sequences = tokenizer.texts_to_sequences(x_train)
print(train_sequences[10])

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

[2, 2, 2, 2, 7, 14, 9, 2, 2, 2, 7, 8, 197, 3525, 242, 2, 2, 22, 216, 11760, 242, 381, 84, 645, 332, 216, 314, 3532, 2, 2, 98, 242, 411, 2, 2, 12519, 2, 2, 25, 2, 2, 2, 2, 18, 2, 2, 2, 2, 260, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 13, 9, 2, 2, 2, 7, 8, 100, 2, 2, 10, 2, 2, 113, 1742, 263, 2, 2, 18, 299, 2, 2, 262, 12, 2, 2, 26, 11, 4108, 256, 2, 2, 221, 123, 2, 2, 2, 2, 2, 2, 2, 2, 755, 103, 207, 2, 2, 2, 2, 25, 393, 382, 2, 2, 13033, 1594, 880, 2, 2, 2, 2762, 11, 17397, 79, 25, 2, 2, 10, 3570, 1387, 4025, 138, 2, 2, 17, 21, 102, 2, 2, 13, 86, 2, 2, 13, 798, 2, 2, 13, 3649, 2, 2, 55, 27, 2, 2, 2, 2, 2, 2, 2, 7, 13, 9, 2, 2, 2, 2, 2, 7, 15, 20, 10, 626, 2, 2, 212, 499, 216, 505, 84, 588, 1, 8, 36, 12, 20, 6929, 126, 223, 4123, 8, 673, 353, 206, 49, 1792, 2171, 1387, 772, 364, 2, 2, 10, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 13, 9, 2, 2, 2, 7, 8, 286, 10, 2, 2, 12, 349, 1379, 24, 60, 2, 2, 676, 296, 2, 2, 2072, 208, 2, 2, 293, 81, 691, 2, 2, 1587, 59, 960, 2348, 22, 898, 470, 2, 2, 43, 570, 1331,

12996
500
1244
500
4083
500


In [5]:
print(train_padded[10])

[    2     2     2     2     7    14     9     2     2     2     7     8
   197  3525   242     2     2    22   216 11760   242   381    84   645
   332   216   314  3532     2     2    98   242   411     2     2 12519
     2     2    25     2     2     2     2    18     2     2     2     2
   260     2     2     2     2     2     2     2     2     2     2     2
     7    13     9     2     2     2     7     8   100     2     2    10
     2     2   113  1742   263     2     2    18   299     2     2   262
    12     2     2    26    11  4108   256     2     2   221   123     2
     2     2     2     2     2     2     2   755   103   207     2     2
     2     2    25   393   382     2     2 13033  1594   880     2     2
     2  2762    11 17397    79    25     2     2    10  3570  1387  4025
   138     2     2    17    21   102     2     2    13    86     2     2
    13   798     2     2    13  3649     2     2    55    27     2     2
     2     2     2     2     2     7    13     9   

In [6]:
validation_sequences = tokenizer.texts_to_sequences(x_test)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

4167
(4167, 500)


In [7]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim),
    layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    layers.Dense(embedding_dim, activation='relu'),
    layers.Dense(6, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         6400000   
                                                                 
 bidirectional (Bidirectiona  (None, None, 512)        1050624   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 512)              1574912   
 nal)                                                            
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 9,158,406
Trainable params: 9,158,406
Non-

In [8]:
callbacks = [keras.callbacks.ModelCheckpoint('zomato_1.keras', save_best_only=True),
             keras.callbacks.EarlyStopping(patience=3)]

In [9]:
history = model.fit(train_padded, y_train, epochs=25, validation_data=(validation_padded, y_test), callbacks=callbacks)

Epoch 1/25
   3/1172 [..............................] - ETA: 4:06:54 - loss: 1.6420 - accuracy: 0.4792

KeyboardInterrupt: 

In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

    
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")